#### This script works on transform the seq2seq model to a graphical model using antra

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
import json
from model import *
from ReaSCAN_dataset import *
import torch.nn.functional as F
import torch
from antra.antra import *
def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter
if isnotebook():
    device = torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
# setting up the seeds.
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

Initialize ReaSCAN dataset to load config of the model

In [2]:
data_directory = "../../../data-files/ReaSCAN-Simple/"
data_file = "data-compositional-splits.txt"
input_vocab_file = "input_vocabulary.txt"
target_vocab_file = "target_vocabulary.txt"
dataset = ReaSCANDataset(
    json.load(open(os.path.join(data_directory, data_file), "r")), 
    data_directory, split="train",
    input_vocabulary_file=input_vocab_file,
    target_vocabulary_file=target_vocab_file,
    generate_vocabulary=False,
    k=0,
)
# Loading a couple of example from ReaSCAN.
dataset.read_dataset(
    max_examples=10,
    simple_situation_representation=True
)

2021-07-22 01:49 Formulating the dataset from the passed in json file...
2021-07-22 01:49 Loading vocabularies...
2021-07-22 01:49 Done loading vocabularies.
2021-07-22 01:49 Converting dataset to tensors...


Define computatinal graph of our model

In [3]:
def _generate_lstm_step_fxn(step_module, i):
    """ 
    Generate a function for a layer in lstm.
    """

    def _lstm_step_fxn(hidden_states):
        (output, hidden, context_situation, attention_weights_commands,
         attention_weights_situations) = step_module(
            hidden_states["input_tokens_sorted"][:, i], 
            hidden_states["hidden"], 
            hidden_states["projected_keys_textual"], 
            hidden_states["commands_lengths"], 
            hidden_states["projected_keys_visual"],
        )
        hidden_states["hidden"] = hidden
        hidden_states["return_lstm_output"] += [output.unsqueeze(0)]
        hidden_states["return_attention_weights"] += [attention_weights_situations.unsqueeze(0)]
        
        return hidden_states

    return _lstm_step_fxn

def generate_compute_graph(model):
    
    
    ####################
    #
    # Input preparation.
    #
    ####################
    """
    Command Inputs.
    """
    command_world_inputs = ["commands_input", "commands_lengths"]
    command_world_input_leaves = [
        GraphNode.leaf(name=name, use_default=True, default_value=None) 
        for name in command_world_inputs
    ]
    @GraphNode(*command_world_input_leaves, cache_results=False)
    def command_input_preparation(
        commands_input, commands_lengths,
    ):
        input_dict = {
            "commands_input": commands_input,
            "commands_lengths": commands_lengths,
        }
        # We may not need the following fields. But we leave it here in case we need these
        # to generate other inputs.
        batch_size = input_dict["commands_input"].shape[0]
        device = input_dict["commands_input"].device
        return input_dict
    
    """
    Situation Inputs.
    """
    situation_inputs = ["situations_input"]
    situation_input_leaves = [
        GraphNode.leaf(name=name, use_default=True, default_value=None) 
        for name in situation_inputs
    ]
    @GraphNode(*situation_input_leaves, cache_results=False)
    def situation_input_preparation(
        situations_input,
    ):
        input_dict = {
            "situations_input": situations_input,
        }
        return input_dict
        
    """
    Target Inputs
    """
    target_sequence_inputs = ["target_batch", "target_lengths"]
    target_sequence_input_leaves = [
        GraphNode.leaf(name=name, use_default=True, default_value=None) 
        for name in target_sequence_inputs
    ]
    @GraphNode(*target_sequence_input_leaves, cache_results=False)
    def target_sequence_input_preparation(
        target_batch, target_lengths
    ):
        input_dict = {
            "target_batch": target_batch,
            "target_lengths": target_lengths,
        }
        return input_dict
    
    ####################
    #
    # Input encoding.
    #
    ####################
    """
    Situation Encoding.
    """
    @GraphNode(situation_input_preparation)
    def situation_encode(input_dict):
        encoded_image = model.situation_encoder(
            input_images=input_dict["situations_input"]
        )
        return encoded_image
    
    """
    Language Encoding.
    """
    @GraphNode(command_input_preparation)
    def command_input_encode(input_dict):
        hidden, encoder_outputs = model.encoder(
            input_batch=input_dict["commands_input"], 
            input_lengths=input_dict["commands_lengths"],
        )
        output_dict = {
            "command_hidden" : hidden,
            "command_encoder_outputs" : encoder_outputs["encoder_outputs"],
            "command_sequence_lengths" : encoder_outputs["sequence_lengths"],
        }
        return output_dict
    
    ####################
    #
    # Decoding.
    #
    ####################
    """
    Preparation of Decoding Data structure.
    """
    @GraphNode(command_input_encode, situation_encode, target_sequence_input_preparation)
    def decode_input_preparation(c_encode, s_encode, target_sequence):
        """
        The decoding step can be represented as:
        h_T = f(h_T-1, C)
        where h_i is the recurring hidden states, and C
        is the static state representations.
        
        In this function, we want to abstract the C.
        """
        
        initial_hidden = model.attention_decoder.initialize_hidden(
            model.tanh(model.enc_hidden_to_dec_hidden(c_encode["command_hidden"])))
        
        """
        Renaming.
        """
        input_tokens, input_lengths = target_sequence["target_batch"], target_sequence["target_lengths"]
        init_hidden = initial_hidden
        encoded_commands = c_encode["command_encoder_outputs"]
        commands_lengths = c_encode["command_sequence_lengths"]
        encoded_situations = s_encode
        
        """
        Reshaping as well as getting the context-guided attention weights.
        """
        batch_size, max_time = input_tokens.size()
        # Sort the sequences by length in descending order
        input_lengths = torch.tensor(input_lengths, dtype=torch.long, device=device)
        input_lengths, perm_idx = torch.sort(input_lengths, descending=True)
        input_tokens_sorted = input_tokens.index_select(dim=0, index=perm_idx)
        initial_h, initial_c = init_hidden
        hidden = (initial_h.index_select(dim=1, index=perm_idx),
                  initial_c.index_select(dim=1, index=perm_idx))
        encoded_commands = encoded_commands.index_select(dim=1, index=perm_idx)
        commands_lengths = torch.tensor(commands_lengths, device=device)
        commands_lengths = commands_lengths.index_select(dim=0, index=perm_idx)
        encoded_situations = encoded_situations.index_select(dim=0, index=perm_idx)

        # For efficiency
        projected_keys_visual = model.visual_attention.key_layer(
            encoded_situations)  # [batch_size, situation_length, dec_hidden_dim]
        projected_keys_textual = model.textual_attention.key_layer(
            encoded_commands)  # [max_input_length, batch_size, dec_hidden_dim]
        
        return {
            "return_lstm_output":[],
            "return_attention_weights":[],
            "hidden":hidden,
            "input_tokens_sorted":input_tokens_sorted,
            "projected_keys_textual":projected_keys_textual,
            "commands_lengths":commands_lengths,
            "projected_keys_visual":projected_keys_visual,
            "perm_idx":perm_idx,
            "seq_lengths":input_lengths,
        }

    hidden_layer = decode_input_preparation
    """
    Here, we set to a static bound of decoding steps.
    """
    max_time = 4
    for i in range(max_time):
        f = _generate_lstm_step_fxn(model.attention_decoder.forward_step, i)
        hidden_layer = GraphNode(hidden_layer,
                                 name=f"lstm_step_{i}",
                                 forward=f)
    """
    Formulating outputs.
    """
    @GraphNode(hidden_layer)
    def output_preparation(hidden_states):
        hidden_states["return_lstm_output"] = torch.cat(
            hidden_states["return_lstm_output"], dim=0)
        hidden_states["return_attention_weights"] = torch.cat(
            hidden_states["return_attention_weights"], dim=0)
        
        _, unperm_idx = hidden_states["perm_idx"].sort(0)
        hidden_states["return_lstm_output"] = hidden_states["return_lstm_output"].index_select(dim=1, index=unperm_idx)  # [max_time, batch_size, output_size]
        hidden_states["seq_lengths"] = hidden_states["seq_lengths"][unperm_idx].tolist()
        hidden_states["return_attention_weights"] = hidden_states["return_attention_weights"].index_select(dim=1, index=unperm_idx)
        
        decoder_output_batched = hidden_states["return_lstm_output"]
        context_situation = hidden_states["return_attention_weights"]
        decoder_output_batched = F.log_softmax(decoder_output_batched, dim=-1)
        
        if model.auxiliary_task:
            pass # Not implemented yet.
        else:
            target_position_scores = torch.zeros(1), torch.zeros(1)
            # We are not returning this as well, since it is not used...
        
        return decoder_output_batched.transpose(0, 1) # [batch_size, max_target_seq_length, target_vocabulary_size]
    
    root = output_preparation # TODO: removing this and continue.
    
    return root
    
class ReaSCANMultiModalLSTMCompGraph(ComputationGraph):
    def __init__(self, model: torch.nn.Module):
        self.model = model
        root = generate_compute_graph(model)

        super().__init__(root)

Loading model to the computational graph

In [4]:
model = Model(
    input_vocabulary_size=dataset.input_vocabulary_size,
    target_vocabulary_size=dataset.target_vocabulary_size,
    num_cnn_channels=dataset.image_channels,
    input_padding_idx=dataset.input_vocabulary.pad_idx,
    target_pad_idx=dataset.target_vocabulary.pad_idx,
    target_eos_idx=dataset.target_vocabulary.eos_idx,
    # language encoder config
    embedding_dimension=25,
    encoder_hidden_size=100,
    num_encoder_layers=1,
    encoder_dropout_p=0.3,
    encoder_bidirectional=True,
    # world encoder config
    simple_situation_representation=True,
    cnn_hidden_num_channels=50,
    cnn_kernel_size=7,
    cnn_dropout_p=0.1,
    auxiliary_task=False,
    # decoder config
    num_decoder_layers=1,
    attention_type="bahdanau",
    decoder_dropout_p=0.3,
    decoder_hidden_size=100,
    conditional_attention=True,
    output_directory="../../../saved_models/ReaSCAN-Simple/"
)
model.eval()
model.to(device)
g = ReaSCANMultiModalLSTMCompGraph(
     model=model,
)

/dfs/user/wuzhengx/tool-chain/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Loading some examples to verify

In [5]:
for (input_batch, input_lengths, _, situation_batch, _, target_batch,
     target_lengths, agent_positions, target_positions) in dataset.get_data_iterator(batch_size=2):
#     target_scores, target_position_scores = model(
#         commands_input=input_batch, commands_lengths=input_lengths,
#         situations_input=situation_batch, target_batch=target_batch,
#         target_lengths=target_lengths
#     )
    # print(target_scores)
    break

In [6]:
input_batch

tensor([[ 1,  3,  4,  5,  6,  7,  8,  9, 10,  2],
        [ 1,  3,  4,  5, 11, 12,  2,  0,  0,  0]])

In [7]:
input_dict = {
    "commands_input": input_batch, 
    "commands_lengths": input_lengths,
    "situations_input": situation_batch,
    "target_batch": target_batch,
    "target_lengths": target_lengths,
}
all_in = GraphInput(input_dict, batched=True, batch_dim=0)

In [8]:
target_scores = g.compute_node("command_input_encode", all_in)

((tensor([[-2.1942e-01,  3.5031e-01,  2.8302e-01,  6.3546e-02, -3.9658e-02,
          8.2039e-02, -2.6031e-01,  1.1407e-01,  3.5423e-02, -7.3205e-02,
          2.2751e-02,  6.2365e-02, -5.6642e-02,  5.4872e-02,  1.8291e-01,
          4.5804e-02, -1.5299e-01, -8.8536e-02,  1.8954e-01,  3.9032e-02,
         -8.3567e-03,  1.2233e-02, -1.3982e-01, -1.5779e-03,  6.6031e-02,
          1.9188e-01, -8.6329e-02,  1.6921e-01, -2.5367e-01, -1.3325e-01,
          1.3773e-01, -1.4950e-01, -2.3418e-02, -9.7508e-02,  6.1760e-02,
          1.4037e-02,  3.7396e-02,  1.9038e-01, -3.8300e-02,  3.8696e-02,
          6.4354e-02, -1.0324e-01,  1.3277e-01, -5.0206e-02,  2.3116e-02,
         -2.3093e-01, -6.3682e-02,  1.5918e-01,  3.4607e-02, -2.0302e-01,
         -6.9659e-02,  1.5170e-01, -5.5256e-02,  3.2395e-02,  1.2660e-01,
         -1.5917e-03,  2.2692e-01,  6.4513e-02,  8.0817e-02,  8.7244e-02,
          7.2151e-02,  1.5747e-01, -1.2346e-01, -4.0435e-02, -9.8149e-02,
         -7.3107e-02,  2.1308e-01, -

In [11]:
g.nodes["command_input_encode"].base_cache

{(('commands_input', (1, 3, 4, 5, 6, 7, 8, 9, 10, 2)),
  ('commands_lengths', 10.0),
  ('situations_input',
   (((0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0),
     (0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0),
     (0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0),
     (0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0),
     (0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.

In [9]:
target_scores

{'command_hidden': tensor([[-2.1942e-01,  3.5031e-01,  2.8302e-01,  6.3546e-02, -3.9658e-02,
           8.2039e-02, -2.6031e-01,  1.1407e-01,  3.5423e-02, -7.3205e-02,
           2.2751e-02,  6.2365e-02, -5.6642e-02,  5.4872e-02,  1.8291e-01,
           4.5804e-02, -1.5299e-01, -8.8536e-02,  1.8954e-01,  3.9032e-02,
          -8.3567e-03,  1.2233e-02, -1.3982e-01, -1.5779e-03,  6.6031e-02,
           1.9188e-01, -8.6329e-02,  1.6921e-01, -2.5367e-01, -1.3325e-01,
           1.3773e-01, -1.4950e-01, -2.3418e-02, -9.7508e-02,  6.1760e-02,
           1.4037e-02,  3.7396e-02,  1.9038e-01, -3.8300e-02,  3.8696e-02,
           6.4354e-02, -1.0324e-01,  1.3277e-01, -5.0206e-02,  2.3116e-02,
          -2.3093e-01, -6.3682e-02,  1.5918e-01,  3.4607e-02, -2.0302e-01,
          -6.9659e-02,  1.5170e-01, -5.5256e-02,  3.2395e-02,  1.2660e-01,
          -1.5917e-03,  2.2692e-01,  6.4513e-02,  8.0817e-02,  8.7244e-02,
           7.2151e-02,  1.5747e-01, -1.2346e-01, -4.0435e-02, -9.8149e-02,
       

In [10]:
# try some basic interventions.
g.compute_node("decode_input_preparation", all_in)

['return_lstm_output']


['return_lstm_output']